In [49]:
import torch
from src.functions import *

print(f"Torch Version {torch.__version__}")

std, mean = torch.load('./models/ChatTTS/asset/spk_stat.pt').chunk(2)

# 空字符串为随机，将会保存在 rand.pt 中
# 否则调用预设
spk_emb_type = ""
spk_emb_save_dir = './models/spk_emb'

if len(spk_emb_type) == 0 or spk_emb_type == "rand":
	rand_spk_emb = torch.randn(768) * std + mean
	spk_emb = rand_spk_emb
	torch.save(spk_emb, os.path.join(spk_emb_save_dir, 'rand.pt')) # save in tmp
	print(f'Speaker: Random')
else:
	spk_emb_save_file_path = os.path.join(spk_emb_save_dir, f'{spk_emb_type}.pt')
	if os.path.exists(spk_emb_save_file_path):
		spk_emb = torch.load(spk_emb_save_file_path) # load speaker embedding
		print(f'Speaker: {spk_emb_type}')
	else:
		print(f"No such speaker: {spk_emb_type}")

params_infer_code = {
	'spk_emb': spk_emb, # add sampled speaker
	'temperature': .3, # using custom temperature
	'top_P': 0.7, # top P decode
	'top_K': 20, # top K decode
}

params_refine_text = {
	'prompt': '[oral_2][laugh_0][break_6]'
}

device = "cpu"
# if torch.backends.mps.is_available() and torch.backends.mps.is_built():
# 	device = "mps"

print(f"Device: {device}")

Torch Version 2.4.0.dev20240529
Speaker: Random
Device: cpu


In [ ]:
import ChatTTS

chat = ChatTTS.Chat()
chat.load_models(
	source = "local",
	local_path = "./models/ChatTTS",
	compile = True,
	device = device
)

In [48]:
# Sound Test

test_texts = [
	"骑士团的大家，有没有认真工作呢?",
	"想要留住雪花，但在手心里它只会融化得更快。"
]

test_wavs = chat.infer(
	solve_texts(test_texts),
	params_refine_text=params_refine_text,
	params_infer_code=params_infer_code
)

display_audio(audios=get_audios(test_wavs), texts=test_texts)

INFO:ChatTTS.core:All initialized.
 11%|█▏        | 233/2048 [00:44<05:50,  5.18it/s]



骑士团的大家，有没有认真工作呢?



想要留住雪花，但在手心里它只会融化得更快。


In [42]:
texts = [
	"但是，总经理人格类型的人不会单打独斗，他们希望自己的可靠性和职业道德能够有所回报。这种人格类型的人言出必行，如果合作伙伴或下属的无能懒惰甚至是欺诈行为威胁到承诺的兑现，他们绝不会压抑怒火。他们因此落下了不懂变通的名声，但这并不是因为他们顽固不化，而是因为他们真的相信这些价值观是社会良性运行的基础。"
]

wavs = chat.infer(
	solve_texts(texts),
	params_refine_text=params_refine_text,
	params_infer_code=params_infer_code
)

INFO:ChatTTS.core:All initialized.
 62%|██████▏   | 1261/2048 [01:33<00:58, 13.44it/s]


In [44]:
display_audio(audios=get_audios(wavs), texts=texts)


但是，总经理人格类型的人不会单打独斗，他们希望自己的可靠性和职业道德能够有所回报。这种人格类型的人言出必行，如果合作伙伴或下属的无能懒惰甚至是欺诈行为威胁到承诺的兑现，他们绝不会压抑怒火。他们因此落下了不懂变通的名声，但这并不是因为他们顽固不化，而是因为他们真的相信这些价值观是社会良性运行的基础。


In [ ]:
save_audios(get_audios(wavs))